In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
            .appName('test-session')\
            .master('local[*]')\
            .config('spark.driver.memory', '24G')\
            .getOrCreate()

print(spark)

In [ ]:
'''
parquet로 저장한 tier1 데이터의 스키마중에 timestamp값 명시해야함
이전 dmc데이터 처리를 그대로 따라하긴 했는데, 생각해보니 실시간으로 쌓이는 데이터를 시간단위로 쪼갠것이
서드파티 데이터처리에 맞춘것인지 아님 데이터 단위를 직관적으로 최소화해 쪼갠것인지 구분을 못하겠음..
로우데이터->표준화->필요에따른 그룹별 데이터

표준화시킨 데이터를 s3에 올려서 emr spark로 돌려보면 좋을것같다
700mb * 30일 * 12달 = 300GB~400GB  -> 1달 23000원 정도? 
emr spark 사용하는 비용은.... 

로컬에서 테스트 돌려보는건 좋은데 클러스터 상황에서 어떤 퍼포먼스가 나오는지 알수가 없네..
일단... 파일전체를 핸들링하는건지 구분하기 어려운데, timestamp를 제대로 처리하지 못하면 곤란함... 
* 일단위로 재처리해야할것같음

'''

In [6]:
# schema 확인
data_path = '/home/data/data-warehouse/tier1/event/date=2021-01-01/*'
df = spark.read.format('parquet') \
    .option('header',True) \
    .option('compression', 'gzip') \
    .load(data_path)    
df.printSchema()
df.show(1,False)

root
 |-- tracking: string (nullable = true)
 |-- trackingId: string (nullable = true)
 |-- trackingEventCode: string (nullable = true)
 |-- cid: string (nullable = true)
 |-- osTypeCode: string (nullable = true)
 |-- logTimeStamp: string (nullable = true)
 |-- eventTimeStamp: string (nullable = true)
 |-- eventDate: date (nullable = true)
 |-- eventHour: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- contentId: string (nullable = true)
 |-- contentName: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- activityParam: string (nullable = true)
 |-- attributed: string (nullable = true)
 |-- latdAdvertisingPartnerName: string (nullable = true)
 |-- hour: string (nullable = true)

+--------+-------------------+-----------------+------------------------------------+----------+-------------------+-------------------+----------+------

In [2]:
from pyspark.sql.functions import *
# 월별매출
data_path = '/home/data/data-warehouse/tier1_temp/event/date=2021-01-01/*'
spark.read.format('parquet') \
        .option('header',True) \
        .option('compression', 'gzip') \
        .load(data_path)\
        .createOrReplaceTempView('t_master')


AnalysisException: Path does not exist: file:/home/data/data-warehouse/tier1_temp/event/date=2021-01-01/*

In [ ]:
sql_temp = """
    select 
        date(eventTimeStamp)
    from t_master
    where 1=1
        and date(eventTimeStamp) BETWEEN date('2021-01-01') AND date('2021-01-02')        
"""

sql = """
    select        
        trackingId,
        osTypeCode,
        count(amount) as purchased_count,
        sum(value) as total_value,
        sum(amount) as total_amount,
        avg(amount) as avg_amount,
        eventTimeStamp

    from t_master
    where 1=1
        and date(eventTimeStamp) BETWEEN date('2021-01-01') AND date('2021-01-02')
    group by trackingId, osTypeCode, eventTimeStamp
    order by trackingId, osTypeCode, eventTimeStamp
"""

# processDate BETWEEN STR_TO_DATE('2021-01-01', '%Y-%m-%d') AND STR_TO_DATE('2021-02-01', '%Y-%m-%d')
# processDate BETWEEN date_format(date_add(now(), INTERVAL -90 DAY)), "%Y-%m-%d") AND date_format(date_add(now(), INTERVAL -1 DAY)), "%Y-%m-%d")
# spark.sql(sql).show(5)

        # cast(eventTimeStamp/1000 as timestamp) as eventDate,
        # from_unixtime(cast(eventTimeStamp/1000 as bigint), 'yyyy-MM-dd HH:mm:ss') as eventDate2



spark.sql(sql).show(5,False)

In [2]:
data_path = '/home/data/data-warehouse/tier1/event/date=2021-01-01/*'
spark.read.format('parquet') \
    .option('header',True) \
    .option('compression', 'gzip') \
    .load(data_path)\
    .createOrReplaceTempView('t_master')

In [ ]:
spark.sql('select * from t_master').show(1)

In [4]:
# t_app_event_purchase
# 구매이벤트 한정해야하나?
sql = '''
    SELECT 
        trackingid          as app_name,
        osTypeCode          as os_typecode,
        cid                 as c_id,
        contentId           as content_id,
        SUM(quantity)       as quantity_total,
        SUM(amount)         as amount_total,
        date(eventTimeStamp)           as event_date,
        year(eventTimeStamp)     as event_year,
        month(eventTimeStamp)    as event_month,
        day(eventTimeStamp)      as event_day

    FROM t_master
    WHERE 1=1
        AND amount > 0
        AND trackingEventCode = 'TPD004'
    GROUP BY trackingid, osTypeCode, date(eventTimeStamp), cid, contentId
    ORDER BY trackingid, date(eventTimeStamp)
'''

spark.sql(sql).show(10,False)

+--------+-----------+------------------------------------+----------+--------------+------------+----------+----------+-----------+---------+
|app_name|os_typecode|c_id                                |content_id|quantity_total|amount_total|event_date|event_year|event_month|event_day|
+--------+-----------+------------------------------------+----------+--------------+------------+----------+----------+-----------+---------+
|11st    |ATC001     |58c49cdc-d250-4ea4-8b6a-6657002495bd|2160134712|1             |9900        |2021-01-01|2021      |1          |1        |
|11st    |ATC001     |f1afaef8-186d-4002-b63b-256b908c02f2|2708031905|1             |30710       |2021-01-01|2021      |1          |1        |
|11st    |ATC001     |0ead5d65-b3fd-414b-b820-7c2468c6d3f3|2981189331|1             |3230        |2021-01-01|2021      |1          |1        |
|11st    |ATC001     |df6efdf8-9348-4863-84fe-ea9d113f7b4b|950320956 |1             |38900       |2021-01-01|2021      |1          |1        |

In [3]:
# t_app_event_user
sql = '''
    SELECT
        trackingid              as app_name,
        cid                     as c_id,
        osTypeCode              as os_typecode,
        COUNT(cid)              as event_count,
        trackingEventCode       as event_typecode,        
        eventDate               as event_date,        
        year(eventDate)         as event_year,
        month(eventDate)        as event_month,
        day(eventDate)          as event_day
    FROM t_master
    WHERE 1=1
        AND trackingid IS NOT NULL
        AND trackingid != 'HNS_DEV'
        AND trim(trackingid) != ''
        AND cid != ''
        AND cid IS NOT NULL
    GROUP BY trackingid, osTypeCode, eventDate, cid, trackingEventCode    
'''
# AND eventDate = ''
spark.sql(sql).show(4,False)


+-------------------+------------------------------------+-----------+-----------+--------------+----------+----------+-----------+---------+
|app_name           |c_id                                |os_typecode|event_count|event_typecode|event_date|event_year|event_month|event_day|
+-------------------+------------------------------------+-----------+-----------+--------------+----------+----------+-----------+---------+
|com.cjoshppingphone|f6afd4bc-1b6d-4021-b56e-23c20c1d09bf|ATC001     |145        |TPD999        |2021-01-01|2021      |1          |1        |
|com.cjoshppingphone|b350709a-9c0e-42ae-844c-b29b10e7db71|ATC001     |50         |TPD001        |2021-01-01|2021      |1          |1        |
|com.cjoshppingphone|4ed0fe07-3267-42ee-8c8b-5d3b58b56f49|ATC001     |69         |TPD001        |2021-01-01|2021      |1          |1        |
|com.cjoshppingphone|5eb645e7-8fa9-49f4-84a7-7721b50a80e4|ATC001     |25         |TPD002        |2021-01-01|2021      |1          |1        |
+-----

In [82]:
# app_매출
data_path = '/home/data/data-warehouse/tier2/purchaseByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')
sql='''
    select
        os_typecode                 as os_typecode,
        event_date                  as event_date,
        count(amount_total)         as purchase_count,
        sum(amount_total)           as amount_total,
        round(avg(amount_total),2)  as amount_avg
    from t_master
    where 1=1 
        and event_date >= '2021-09-01'
        and event_date <= '2022-01-01'
    group by app_name, os_typecode, event_date
    order by app_name, os_typecode, event_date
'''
spark.sql(sql).show(4,False)

+----------+----------+--------------+------------+----------+
|osTypeCode|eventDate |purchase_count|amount_total|amount_avg|
+----------+----------+--------------+------------+----------+
|ATC001    |2021-09-01|54096         |3378433638  |62452.56  |
|ATC001    |2021-09-02|42580         |2566089690  |60265.14  |
|ATC001    |2021-09-03|28878         |1801766094  |62392.34  |
|ATC001    |2021-09-04|28860         |1606136278  |55652.68  |
+----------+----------+--------------+------------+----------+
only showing top 4 rows



In [115]:
# app_매출 이동평균
data_path = '/home/data/data-warehouse/tier2/purchaseByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
    select
        app_name,
        os_typecode,
        event_date,
        amount_total,
        case
            when 7 = count(*)
                over(partition by app_name, os_typecode order by event_date rows between 6 preceding and current row)
            then 
                avg(sum(sumAmount)) 
                over(partition by app_name, os_typecode order by event_date rows between 6 preceding and current row) 
        end as amount_avg_7day
    from t_master
    where 1=1
        and event_date >= '2021-01-01'
        and event_date <= '2021-01-10'
    group by app_name, os_typecode, event_date
    order by app_name, os_typecode, event_date
'''
# 윈도우함수로 7일평균 컬럼 만들기
# avg({col}) over(partition by {col} order by {col} rows between unbounded preceding and current row)
# group by로 묶어주는 경우 적정한 partition을 잡아줘야함
df = spark.sql(sql)
df.withColumn('avg', df.amount_avg_7day.cast('bigint') ).show(10,False)  # avg연산에서 자료형이 바뀌므로 bigint로 변환

+----------+----------+----------+------------+-------------------+----------+
|trackingid|osTypeCode|eventDate |amount_total|amount_avg_7day    |avg       |
+----------+----------+----------+------------+-------------------+----------+
|11st      |ATC001    |2021-01-01|3639494982  |null               |null      |
|11st      |ATC001    |2021-01-02|2826525497  |null               |null      |
|11st      |ATC001    |2021-01-03|3219519010  |null               |null      |
|11st      |ATC001    |2021-01-04|3445656772  |null               |null      |
|11st      |ATC001    |2021-01-05|3126742495  |null               |null      |
|11st      |ATC001    |2021-01-06|2720626938  |null               |null      |
|11st      |ATC001    |2021-01-07|3298286934  |3.182407518285714E9|3182407518|
|11st      |ATC001    |2021-01-08|2795548843  |3.061843784142857E9|3061843784|
|11st      |ATC001    |2021-01-09|2643404848  |3.035683691428571E9|3035683691|
|11st      |ATC001    |2021-01-10|4698175340  |3.246

In [76]:
# app_매출누계
data_path = '/home/data/data-warehouse/tier2/purchaseByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
    select
        app_name,
        os_typecode,
        event_date,
        amount_total,
        sum(sum(amount_total)) 
        over(partition by app_name, os_typecode order by event_date rows between unbounded preceding and current row) as amount_agg
    from t_master
    where 1=1
        and event_date >= '2021-01-01'
        and event_date <= '2021-01-10'
    group by app_name, os_typecode, event_date
    order by app_name, os_typecode, event_date
'''
# 윈도우함수로 누적컬럼 만들기
# sum({col}) over(partition by {col} order by {col} rows between unbounded preceding and current row)
# group by로 묶어주는 경우 적정한 partition을 잡아줘야함
spark.sql(sql).show(5,False)

+----------+----------+----------+------------+-----------+
|trackingid|osTypeCode|eventDate |amount_total|amount_agg |
+----------+----------+----------+------------+-----------+
|11st      |ATC001    |2021-01-01|3639494982  |3639494982 |
|11st      |ATC001    |2021-01-02|2826525497  |6466020479 |
|11st      |ATC001    |2021-01-03|3219519010  |9685539489 |
|11st      |ATC001    |2021-01-04|3445656772  |13131196261|
|11st      |ATC001    |2021-01-05|3126742495  |16257938756|
+----------+----------+----------+------------+-----------+
only showing top 5 rows



In [4]:
# 매출대비
data_path = '/home/data/data-warehouse/tier2/purchaseByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
    select
        app_name,
        os_typecode,
        event_month,
        sum(amount_total),
        lag(sum(amount_total),1) over(partition by app_name, os_typecode order by event_month) as pre_amount,
        sum(amount_total) - lag(sum(amount_total),1) over(partition by app_name, os_typecode order by event_month) as diff_amount
        
    from t_master
    where 1=1
        and app_name != ''
        and year(event_date) = '2021'
    group by app_name, os_typecode, event_month
    order by app_name, os_typecode, event_month
'''
# 년도별로 sum해서 rate만드는건데 년도가 하나밖에 없으니 month기준으로 해보자
spark.sql(sql).show(12,False)

+--------+-----------+-----------+-----------------+------------+------------+
|app_name|os_typecode|event_month|sum(amount_total)|pre_amount  |diff_amount |
+--------+-----------+-----------+-----------------+------------+------------+
|11st    |ATC001     |1          |80751616270      |null        |null        |
|11st    |ATC001     |2          |65205686527      |80751616270 |-15545929743|
|11st    |ATC001     |3          |120917739119     |65205686527 |55712052592 |
|11st    |ATC001     |4          |128184721137     |120917739119|7266982018  |
|11st    |ATC001     |5          |117550160955     |128184721137|-10634560182|
|11st    |ATC001     |6          |93090703142      |117550160955|-24459457813|
|11st    |ATC001     |7          |106874106219     |93090703142 |13783403077 |
|11st    |ATC001     |8          |83644659640      |106874106219|-23229446579|
|11st    |ATC001     |9          |126675916478     |83644659640 |43031256838 |
|11st    |ATC001     |10         |159859037891     |

In [38]:
# 구매 가격대 히스토그램
data_path = '/home/data/data-warehouse/tier1/event/date=2021-05-01/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
with stats as (
    select
        10000000        as max_price
        ,0              as min_price
        ,10000000       as range_price
        ,10             as bucket_num
    
)
,purchase_log_with_bucket as (
    select
        value as price
        ,min_price
        ,value - min_price as diff
        ,1.0 * range_price / bucket_num as bucket_range
        ,floor(1.0 * (value - min_price) / (1.0 * range_price / bucket_num)) + 1 as bucket

    from t_master, stats
    where 1=1
        and value > 100 
        and value <= 10000000
        and value is not null
        and trackingEventCode = 'TPD004' 
        -- and eventDate >= date('2021-06-01') 
        -- and eventDate <= date('2021-06-15')        
)

select 
    bucket
    ,min_price + bucket_range * (bucket-1)  as lower_limit
    ,min_price + bucket_range * bucket      as upper_limit
    ,count(price)                           as num_purchase
    ,sum(price)                             as total_amount

from purchase_log_with_bucket
group by bucket, min_price, bucket_range
order by bucket
'''
# 
# from t_master
#     where 1=1
#         and value > 1 and value is not null
#         and trackingEventCode = 'TPD004' 
#         and eventDate >= date('2021-06-01') 
#         and eventDate <= date('2021-06-15')
spark.sql(sql).show(12,False)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [31]:
# 구매 가격대 데이터 확인
# 3000만원대 가격이 뭔가했더니 hmall 롯데렌터카 상품이 월렌트 00에 총 가격 3000만원 이런식으로 되어있음
# 1,2원대는 위메프인데 뭔질 모르겠다...
data_path = '/home/data/data-warehouse/tier1/event/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql = '''
    select * from t_master
    where 1=1
        and value > 1 and value is not null
        and trackingEventCode = 'TPD004' 
        and eventDate >= date('2021-06-01') 
        and eventDate <= date('2021-06-15')
        and value = 2
    limit 1 
'''
spark.sql(sql).show(1,False)

+--------+-----------+-----------------+---+----------+-------------------+-------------------+----------+---------+--------+----------+-----------+-----+--------+------+--------+-------------+----------+--------------------------+----+
|tracking|trackingId |trackingEventCode|cid|osTypeCode|logTimeStamp       |eventTimeStamp     |eventDate |eventHour|campaign|contentId |contentName|value|quantity|amount|currency|activityParam|attributed|latdAdvertisingPartnerName|hour|
+--------+-----------+-----------------+---+----------+-------------------+-------------------+----------+---------+--------+----------+-----------+-----+--------+------+--------+-------------+----------+--------------------------+----+
|BRANCH  |Wemakeprice|TPD004           |   |ATC002    |2021-06-01 16:07:16|2021-06-01 16:04:17|2021-06-01|16       |        |1520824309|           |2    |100     |200   |KRW     |             |1         |                          |16  |
+--------+-----------+-----------------+---+--------

In [5]:
# app_user_event집계 
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
    select
        app_name,
        c_id
    from t_master
    where 1=1
        and event_date >= date('2021-01-01') 
        and event_date <= date('2021-02-01')
    group by app_name, c_id
'''
spark.sql(sql).show(12,False)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3361, in run_code
    exec(code_obj, self.user_global_ns, self.

ConnectionRefusedError: [Errno 111] Connection refused

In [3]:
# app_user_event집계 
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
with event_day_count_per_user as (
    select
        app_name,
        c_id,
        count(distinct event_date) as event_day_count
    from t_master
    where 1=1
        and event_date >= date('2021-01-01')
        and event_date <= date('2021-03-01')
    group by app_name, c_id
)

select 
    app_name,
    event_day_count,
    count(distinct c_id) as user_count,
    100 * count(distinct c_id) / sum(count(distinct c_id)) over(partition by app_name) as composition_ratio,
    100 * sum(count(distinct c_id)) over(partition by app_name order by event_day_count rows between unbounded preceding and current row) / 
    sum(count(distinct c_id)) over(partition by app_name) as cumulative_ratio

from event_day_count_per_user
group by app_name, event_day_count
order by app_name, event_day_count desc
    
'''

# 뭘 확인하려고 event_day_count로 그룹하는거지?
# event_day_count -> 기간안에서 유저별 액션이 일어난 날짜의 수
# composition_ratio -> 전체유저중 비율
# cumulative_ratio -> 전체유저중 누적비율
spark.sql(sql).show(12,False)

+--------+---------------+----------+---------------------+-----------------+
|app_name|event_day_count|user_count|composition_ratio    |cumulative_ratio |
+--------+---------------+----------+---------------------+-----------------+
|11st    |60             |159       |0.002544600932956327 |100.0            |
|11st    |59             |203       |0.0032487672288687695|99.99745539906705|
|11st    |58             |263       |0.0042089939960221   |99.99420663183818|
|11st    |57             |410       |0.006561549575547761 |99.98999763784215|
|11st    |56             |560       |0.008962116493431089 |99.9834360882666 |
|11st    |55             |770       |0.012322910178467747 |99.97447397177318|
|11st    |54             |1099      |0.017588153618358512 |99.96215106159471|
|11st    |53             |1401      |0.022421295013030276 |99.94456290797635|
|11st    |52             |1944      |0.03111134725576792  |99.92214161296332|
|11st    |51             |2521      |0.04034552799989245  |99.89

In [5]:
# app_user_event 타입별 집계 
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
    select 
        event_hour
        ,app_name
        ,os_typecode
        ,count(distinct c_id) as user_count
        ,count(c_id) as user_access_count
        ,COUNT(CASE WHEN event_typecode = 'TPD008' THEN c_id END) AS install
        ,COUNT(CASE WHEN event_typecode = 'TPD009' THEN c_id END) AS open
        ,COUNT(CASE WHEN event_typecode = 'TPD007'  OR c_id = 'TPD001' THEN event_typecode END) AS search
        ,COUNT(CASE WHEN event_typecode = 'TPD002' THEN c_id END) AS viewcontent
        ,COUNT(CASE WHEN event_typecode = 'TPD003' OR c_id = 'TPD016' THEN event_typecode END) AS addtocart
        ,COUNT(CASE WHEN event_typecode = 'TPD004' THEN c_id END) AS purchase

        ,COUNT(CASE WHEN event_typecode = 'TPD007'  OR event_typecode = 'TPD001' THEN event_typecode END) / COUNT(DISTINCT c_id)  AS search_per_user
        ,COUNT(CASE WHEN event_typecode = 'TPD002' THEN event_typecode END) / COUNT(DISTINCT c_id) AS viewcontent_per_user
        ,COUNT(CASE WHEN event_typecode = 'TPD003' OR event_typecode = 'TPD016' THEN event_typecode END) / COUNT(DISTINCT c_id) AS addtocart_per_user
        ,COUNT(CASE WHEN event_typecode = 'TPD004' THEN event_typecode END) / COUNT(DISTINCT c_id) AS purchase_per_user


    from t_master
    where 1=1
        and event_date >= date('2021-01-01')
        and event_date <= date('2021-03-01')
    group by event_date, event_hour, app_name, os_typecode
    order by event_date desc, event_hour asc, app_name, os_typecode
'''

spark.sql(sql).show(12,False)

+----------+----------------------+-----------+----------+-----------------+-------+-----+------+-----------+---------+--------+-------------------+--------------------+-------------------+--------------------+
|event_hour|app_name              |os_typecode|user_count|user_access_count|install|open |search|viewcontent|addtocart|purchase|search_per_user    |viewcontent_per_user|addtocart_per_user |purchase_per_user   |
+----------+----------------------+-----------+----------+-----------------+-------+-----+------+-----------+---------+--------+-------------------+--------------------+-------------------+--------------------+
|00        |11st                  |ATC001     |38507     |42368            |0      |18055|0     |18993      |3609     |1711    |0.0                |0.4932349962344509  |0.09372321915495883|0.04443347962708079 |
|00        |11st                  |ATC002     |5087      |5698             |0      |1614 |0     |3205       |582      |297     |0.0                |0.630037

In [15]:
# app_user_event 이벤트 벤다이어그램
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
with user_event_flag as (
    select 
        app_name
        ,os_typecode
        ,c_id
        ,SIGN(SUM(CASE WHEN event_typecode = 'TPD002' THEN 1 ELSE 0 END)) AS has_viewcontent
        ,SIGN(SUM(CASE WHEN event_typecode = 'TPD003' THEN 1 ELSE 0 END)) AS has_cart
        ,SIGN(SUM(CASE WHEN event_typecode = 'TPD004' THEN 1 ELSE 0 END)) AS has_purchased

    from t_master
    where 1=1        
        and event_date >= date('2021-01-01')
        and event_date <= date('2021-03-01')
        and event_typecode in ('TPD002','TPD003','TPD004','TPD011')
        and app_name is not null
        and os_typecode is not null

    group by app_name, os_typecode, c_id

), event_view_diagram as (
    select 
        app_name
        ,os_typecode
        ,has_viewcontent
        ,has_cart
        ,has_purchased
        ,count(1) as user_count
    from user_event_flag
    group by app_name, os_typecode, cube(has_viewcontent, has_cart, has_purchased)    
)

select 
    app_name
    ,os_typecode
    -- flag값을 내용으로 치환
    ,case has_viewcontent when 1 then 'viewcontent' when 0 then 'not viewcontent' else 'any' end as has_viewcontent
    ,case has_cart when 1 then 'cart' when 0 then 'not cart' else 'any' end as has_cart
    ,case has_purchased when 1 then 'purchase' when 0 then 'not purchase' else 'any' end as has_purchased
    ,user_count
    ,100 * user_count / 
    -- 모든 액션이 null인 사용자가 전체사용자?
    nullif(
        sum(case when has_viewcontent is null and has_cart is null and has_purchased is null then user_count else 0 end) 
        over(partition by app_name, os_typecode)
    ,0) as ratio
    from event_view_diagram
    order by app_name, os_typecode, has_purchased, has_cart, has_viewcontent
    
'''

# sign -> -1,0,1 출력
# cube -> group by 절에서 컬럼별 계산 가능한 모든 소계와 합계 반환
# 1.flag 테이블을 만들어서 
# 2.cube함수로 묶으면 항목별 flag에 해당하는 유저 카운트 생성
# 3.다시 app,ostype, flag를 가공한 문자열을 통해 항목별 user_count, 비율 구하기
# 모든 액션이 null인 사용자가 전체사용자임? -> 2번 과정에서 셋다 null로 잡히는게 전체 갯수인데 이건 cube함수의 특징
spark.sql(sql).show(50,False)

+--------+-----------+---------------+--------+-------------+----------+-------------------+
|app_name|os_typecode|has_viewcontent|has_cart|has_purchased|user_count|ratio              |
+--------+-----------+---------------+--------+-------------+----------+-------------------+
|11st    |ATC001     |any            |any     |any          |3737362   |100.0              |
|11st    |ATC001     |not viewcontent|any     |any          |75100     |2.0094387431562692 |
|11st    |ATC001     |viewcontent    |any     |any          |3662262   |97.99056125684373  |
|11st    |ATC001     |any            |cart    |any          |1102839   |29.508487537466266 |
|11st    |ATC001     |not viewcontent|cart    |any          |31750     |0.8495296950094746 |
|11st    |ATC001     |viewcontent    |cart    |any          |1071089   |28.658957842456793 |
|11st    |ATC001     |any            |not cart|any          |2634523   |70.49151246253373  |
|11st    |ATC001     |not viewcontent|not cart|any          |43350    

In [32]:
# app_user_event 주간/시간별 이벤트
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
with event_with_dow as (
    select
        app_name
        ,event_date
        ,date_format(event_date, 'E') as dow
        ,event_hour
        ,count(*) as event_count

    from t_master
    where 1=1
        and event_date >= date('2021-01-01')
        and event_date <= date('2021-03-01')
    group by app_name, event_date, event_hour
    order by app_name, event_date, event_hour
)

select 
    app_name
    ,event_hour
    ,sum(case dow when 'Mon' then event_count end) as mon
    ,sum(case dow when 'Tue' then event_count end) as tue
    ,sum(case dow when 'Wed' then event_count end) as wed
    ,sum(case dow when 'Thu' then event_count end) as thu
    ,sum(case dow when 'Fri' then event_count end) as fri
    ,sum(case dow when 'Sat' then event_count end) as sat
    ,sum(case dow when 'Sun' then event_count end) as sun

from event_with_dow
group by app_name, event_hour
order by app_name, event_hour
'''

spark.sql(sql).show(50,False)

+----------------------+----------+------+------+------+------+------+------+------+
|app_name              |event_hour|mon   |tue   |wed   |thu   |fri   |sat   |sun   |
+----------------------+----------+------+------+------+------+------+------+------+
|11st                  |00        |461041|359389|463580|425650|467405|594519|545009|
|11st                  |01        |421295|347173|409566|396132|394853|519686|467458|
|11st                  |02        |449711|406691|416486|403185|411746|505519|463892|
|11st                  |03        |457849|457520|440348|397301|430698|472842|447890|
|11st                  |04        |441061|442848|426782|362674|384968|469942|437498|
|11st                  |05        |458320|437275|416904|369715|373903|470514|441280|
|11st                  |06        |442664|471743|423577|398901|397582|485210|441705|
|11st                  |07        |433150|471438|433906|396861|423953|480464|442653|
|11st                  |08        |446428|457676|445710|400668|45

In [39]:
# app_user_event 주간/시간별 이벤트 확인
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
with table as (
    select 
        app_name
        ,event_hour
    from t_master
    where 1=1
        and app_name = '11st'
        and event_hour = '00'
        and date_format(event_date,'E') = 'Mon'
        and event_date >= date('2021-01-01')
        and event_date <= date('2021-03-01')
)
select count(*) from table
'''

spark.sql(sql).show(50,False)

+--------+
|count(1)|
+--------+
|461041  |
+--------+



In [14]:
# app_user_event 시계열 구매/카트
data_path = '/home/data/data-warehouse/tier1/event/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
with row_event_log as (
    select 
       trackingId           as app_name
       ,osTypeCode          as os_typecode  
       ,eventDate           as event_date
       ,eventTimeStamp      as event_timestamp
       ,case when trackingEventCode in ('TPD011','TPD004') then 'purchase' else 'add_cart' end as event_name
       ,cid                 as c_id
       ,contentId           as content_id
    from t_master
    where 1=1
        and eventDate >= date('2021-06-01')
        and eventDate <= date('2021-08-01')
        and osTypeCode != 'ATC999'
        and trackingEventCode in ('TPD003', case when trackingId in ('홈&쇼핑','hnsmall11') then 'TPD011' else 'TPD004' end)
)

, event_time_stats as (
    select
        app_name
        ,os_typecode
        ,c_id
        ,content_id
        ,min(case event_name when 'add_cart' then event_date end) as add_cart_date
        ,min(case event_name when 'purchase' then event_date end) as purchase_date
        -- 구매시간-카트시간(초)
        ,unix_timestamp(min(case event_name when 'purchase' then event_timestamp end)) - 
         unix_timestamp(min(case event_name when 'add_cart' then event_timestamp end)) as lead_time
    from row_event_log
    group by app_name, os_typecode, c_id, content_id
)

, purchase_lead_time_flag as (
    select
        app_name
        ,os_typecode
        ,c_id
        ,add_cart_date
        ,purchase_date
        ,lead_time
        ,case when (lead_time <= 1.0 * 60 * 60) then 1 else 0 end                 as purchase_1_hour
        ,case when (lead_time <= 6.0 * 60 * 60) then 1 else 0 end                 as purchase_6_hour
        ,case when (lead_time <= 24.0 * 60 * 60) then 1 else 0 end                as purchase_24_hour
        ,case when (lead_time <= 48.0 * 60 * 60) then 1 else 0 end                as purchase_48_hour
        ,case when (lead_time is null or not(lead_time <= 48.0 * 60 * 60)) and (purchase_date is null)
            then 1 else 0 end                                                     as not_purchase
        ,case when (lead_time is null) and (purchase_date is not null)
            then 1 else 0 end                                                     as not_add_cart_purchase
    from event_time_stats
)

select
    app_name
    ,os_typecode
    ,add_cart_date    
    ,count(*)                               as add_cart    
    ,sum(purchase_1_hour)                     as purchase_1_hour
    ,round(avg(purchase_1_hour),2)            as purchase_1_hour_rate
    ,sum(purchase_6_hour)                     as purchase_6_hour
    ,round(avg(purchase_6_hour),2)            as purchase_6_hour_rate
    ,sum(not_purchase)                      as not_purchase
    ,round(avg(not_purchase),2)             as not_purchase_rate  
    ,sum(not_add_cart_purchase)             as not_add_cart_purchase
    ,round(avg(not_add_cart_purchase),2)    as not_add_cart_purchase_rate
from purchase_lead_time_flag
group by app_name, os_typecode, add_cart_date
order by app_name, os_typecode, add_cart_date


'''



# tier1 이벤트 데이터를 통해 시작
# 1.유저별 카트담은시간-구매시간 차이 계산 -> 기간에따라 null값도 있음
# 2.플래그 값으로 변경
# 3.합계와 평균 생성
spark.sql(sql).show(50,False)

+--------+-----------+-------------+--------+---------------+--------------------+---------------+--------------------+------------+-----------------+---------------------+--------------------------+
|app_name|os_typecode|add_cart_date|add_cart|purchase_1_hour|purchase_1_hour_rate|purchase_6_hour|purchase_6_hour_rate|not_purchase|not_purchase_rate|not_add_cart_purchase|not_add_cart_purchase_rate|
+--------+-----------+-------------+--------+---------------+--------------------+---------------+--------------------+------------+-----------------+---------------------+--------------------------+
|11st    |ATC001     |null         |3038908 |0              |0.0                 |0              |0.0                 |0           |0.0              |3038908              |1.0                       |
|11st    |ATC001     |2021-06-01   |120583  |8239           |0.07                |9514           |0.08                |105910      |0.88             |0                    |0.0                       |


In [21]:
# adbrix만 가입이벤트가 구분되어 들어오고있음..
# 가입 후 지속률, 정착율 구하기는 adbrix한정으로 진행됨
# 사용자의 가입일, 가입일+1, event일, last event일  테이블을 생성
# app_user 테이블의 create_date가 등록일 의미하는거였구만. min 값으로 퉁치는데 update는 max. 이렇게 퉁쳐도 되나?
# 그럼이걸 매일 갱신해서 저장하는건가? -> 그런듯
# user 가입일이 있고, user별 액션도 종류별로 있는 상태에서 가입->액션->구매 집계가 이루어져야함
# 이 테이블을 누적으로 쌓아두려면? -> 누적으로 쌓아둘 필요가 있을까?
# 가입이벤트 이전 행동했던 이벤트도 있음
# *최종정리 -> 유저별 기간내 가입이벤트 + 이후 이벤트 목록
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')

sql='''
with register_log as (
    select
        app_name
        ,c_id        
        ,event_typecode        
        ,event_date as register_date

    from t_master
    where 1=1        
        and event_typecode = 'TPD012'
        and c_id != '00000000-0000-0000-0000-000000000000'
        and event_date >= date('2021-01-01')
        and event_date <= date('2021-03-01')
),

register_event_log as (
    select
        m.app_name
        ,m.c_id
        ,m.register_date
        ,t.event_typecode
        ,t.event_date
        
    from register_log as m
    left join t_master as t on t.c_id = m.c_id and t.app_name = m.app_name

    where 1=1
        and t.c_id != '00000000-0000-0000-0000-000000000000'
        and t.event_date >= date('2021-01-01')
        and t.event_date <= date('2021-03-01')
        and t.event_date >= m.register_date    
)

select * from register_event_log order by app_name,c_id,event_date
'''

#  ,case when event_typecode='TPD012' then event_date else null end as register_date

spark.sql(sql).show(20,False)

+----------------------+------------------------------------+-------------+--------------+----------+
|app_name              |c_id                                |register_date|event_typecode|event_date|
+----------------------+------------------------------------+-------------+--------------+----------+
|com.cjmall.CJOShopping|0003A28E-0F40-4B30-B524-1DB40FB09FEB|2021-01-26   |TPD012        |2021-01-26|
|com.cjmall.CJOShopping|0003A28E-0F40-4B30-B524-1DB40FB09FEB|2021-01-26   |TPD999        |2021-01-26|
|com.cjmall.CJOShopping|0003A28E-0F40-4B30-B524-1DB40FB09FEB|2021-01-26   |TPD004        |2021-01-26|
|com.cjmall.CJOShopping|0003A28E-0F40-4B30-B524-1DB40FB09FEB|2021-01-26   |TPD013        |2021-01-26|
|com.cjmall.CJOShopping|00122D4D-AEE4-40AC-A68E-24805C3532D5|2021-01-09   |TPD012        |2021-01-09|
|com.cjmall.CJOShopping|00122D4D-AEE4-40AC-A68E-24805C3532D5|2021-01-09   |TPD001        |2021-01-09|
|com.cjmall.CJOShopping|00122D4D-AEE4-40AC-A68E-24805C3532D5|2021-01-09   |TPD002 

In [44]:
# app_user_event_retention 
data_path = '/home/data/data-warehouse/tier2/eventByUser/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).createOrReplaceTempView('t_master')
sql='''
with repeat_interval(index_name, interval_date) as (
    values('01 day repeat', 1)
    ,('02 day repeat', 2)
    ,('03 day repeat', 3)
    ,('04 day repeat', 4)
    ,('05 day repeat', 5)
    ,('06 day repeat', 6)
    ,('07 day repeat', 7)        
),

-- 기간내 회원등록 이벤트
register_log as (
    select
        app_name
        ,c_id        
        ,event_typecode        
        ,event_date as register_date

    from t_master
    where 1=1        
        and event_typecode = 'TPD012'                       -- 등록이벤트
        and c_id != '00000000-0000-0000-0000-000000000000'  -- cid 예외처리
        and event_date >= date('2021-01-01')                -- 기간설정
        and event_date <= date('2021-03-01')
),

-- 등록이벤트가 있는 id를 left join한 이벤트 리스트
register_event_log_with_index_date as (
    select
        m.app_name
        ,m.c_id
        ,m.register_date
        ,t1.event_typecode
        ,t1.event_date
        ,max(t1.event_date) over(partition by m.app_name, m.c_id) as latest_date
        ,t2.index_name
        ,date_add(m.register_date, t2.interval_date) as index_date -- 크로스조인으로 확장 용이하게
    
    from register_log as m 
    left join t_master as t1 on t1.c_id = m.c_id and t1.app_name = m.app_name
    cross join repeat_interval as t2

    where 1=1
        and t1.c_id != '00000000-0000-0000-0000-000000000000'
        and t1.event_date >= date('2021-01-01')
        and t1.event_date <= date('2021-03-01')
        and t1.event_date >= m.register_date    
),

-- 플래그생성
user_action_flag as (
    select
        app_name
        ,c_id
        ,register_date
        ,index_name
        ,sign(
            sum(
                case when index_date <= latest_date then
                    case when index_date = event_date then 1 else 0 end
                end
            )
        ) as index_date_action
    from register_event_log_with_index_date
    group by app_name, c_id, register_date, index_name, index_date

)

-- 지속율 평균%
select
    app_name
    ,register_date
    ,index_name
    ,round(avg(100.0 * index_date_action),2) as repeat_rate -- 지속율
from user_action_flag
group by app_name, register_date, index_name
order by app_name, register_date, index_name
'''

#  ,case when event_typecode='TPD012' then event_date else null end as register_date

spark.sql(sql).show(20,False)

+----------------------+-------------+-------------+-----------+
|app_name              |register_date|index_name   |repeat_rate|
+----------------------+-------------+-------------+-----------+
|com.cjmall.CJOShopping|2021-01-01   |01 day repeat|38.89      |
|com.cjmall.CJOShopping|2021-01-01   |02 day repeat|18.75      |
|com.cjmall.CJOShopping|2021-01-01   |03 day repeat|13.33      |
|com.cjmall.CJOShopping|2021-01-01   |04 day repeat|21.43      |
|com.cjmall.CJOShopping|2021-01-01   |05 day repeat|16.67      |
|com.cjmall.CJOShopping|2021-01-01   |06 day repeat|27.27      |
|com.cjmall.CJOShopping|2021-01-01   |07 day repeat|27.27      |
|com.cjmall.CJOShopping|2021-01-02   |01 day repeat|27.59      |
|com.cjmall.CJOShopping|2021-01-02   |02 day repeat|23.08      |
|com.cjmall.CJOShopping|2021-01-02   |03 day repeat|34.78      |
|com.cjmall.CJOShopping|2021-01-02   |04 day repeat|30.0       |
|com.cjmall.CJOShopping|2021-01-02   |05 day repeat|10.53      |
|com.cjmall.CJOShopping|2